In [30]:
import pandas as pd
from source import data_import
from source.constants import CHANNELS

print(f"loaded channel names: {CHANNELS}")

loaded channel names: ['F4-C4', 'F3-C3', 'FT9-FT10', 'FZ-CZ', 'F7-T7', 'FP2-F4', 'T8-P8-1', 'T8-P8-0', 'FP1-F3', 'CZ-PZ']


In [31]:
# load data
df_pp = data_import.load_pyarrow(file_name="preprocessed_df")

/home/weasel/reps/ai-seizure-detectives/source/../data/preprocessed_df.arrow was loaded.


## Feature Extraction

In [32]:
df_features = df_pp.groupby(['seizure_id', 'segment_id']).agg(
    {C:['mean', 'std'] for C in CHANNELS} | 
    {'is_seizure': 'first'}) # just taking first element for target column
df_features.head()

channel                   F4-C4                F3-C3             FT9-FT10  \
                           mean        std      mean        std      mean   
seizure_id segment_id                                                       
0          1           0.642823  25.278278 -0.136380  28.577184  0.109270   
           2           0.459926  24.908361  0.607864  22.789379  0.984358   
           3          -0.336786  25.078313  0.020483  24.208610 -0.237448   
           4           0.571550  23.796227  0.170771  27.059930 -0.130328   
           5          -0.156618  28.030262  0.019398  28.681243  0.157825   

channel                              FZ-CZ                F7-T7             \
                             std      mean        std      mean        std   
seizure_id segment_id                                                        
0          1           27.872715  0.072943  40.156441  1.195346  20.135111   
           2           23.485323  0.457697  32.960306  0.657212  18.265025   
           3           20.733267 -0.281956  32.829720 -0.433444  17.771789   
           4           23.920272  0.457751  34.066019  0.129564  17.220239   
           5           25.146513  0.097109  42.098599  0.096355  17.888231   

channel                ...     FP2-F4   T8-P8-1              T8-P8-0  \
                       ...        std      mean        std      mean   
seizure_id segment_id  ...                                             
0          1           ...  25.380978  0.365832  22.583062  0.365832   
           2           ...  23.324690  0.455336  21.314656  0.455336   
           3           ...  18.870577 -0.051306  20.050884 -0.051306   
           4           ...  22.235792  0.214385  19.831823  0.214385   
           5           ...  29.556387 -0.188304  21.520493 -0.188304   

channel                             FP1-F3                CZ-PZ             \
                             std      mean        std      mean        std   
seizure_id segment_id                                                        
0          1           22.583062 -0.383942  26.546966  0.758870  35.447713   
           2           21.314656  1.242494  25.584776  0.040398  33.861154   
           3           20.050884 -0.092616  22.492845  0.139786  30.472674   
           4           19.831823  0.136081  24.970415  0.347514  30.803236   
           5           21.520493 -0.279403  28.959603 -0.032786  39.120852   

channel               is_seizure  
                           first  
seizure_id segment_id             
0          1               False  
           2               False  
           3               False  
           4               False  
           5               False  

[5 rows x 21 columns]

## Flatten Dataframe

In [33]:
# joining column names with agg functions, but leaving target 'is_seizure' column as 'is_seizure'.
df_features.columns = ['_'.join(col).strip() for col in df_features.columns.values if 'is_seizure' != col[0]] + ['is_seizure']
df_features.reset_index(inplace=True)
df_features.head(20)

,seizure_id,segment_id,F4-C4_mean,F4-C4_std,F3-C3_mean,F3-C3_std,FT9-FT10_mean,FT9-FT10_std,FZ-CZ_mean,FZ-CZ_std,...,FP2-F4_std,T8-P8-1_mean,T8-P8-1_std,T8-P8-0_mean,T8-P8-0_std,FP1-F3_mean,FP1-F3_std,CZ-PZ_mean,CZ-PZ_std,is_seizure
0,0,1,0.642823,25.278278,-0.136380,28.577184,0.109270,27.872715,0.072943,40.156441,...,25.380978,0.365832,22.583062,0.365832,22.583062,-0.383942,26.546966,0.758870,35.447713,False
1,0,2,0.459926,24.908361,0.607864,22.789379,0.984358,23.485323,0.457697,32.960306,...,23.324690,0.455336,21.314656,0.455336,21.314656,1.242494,25.584776,0.040398,33.861154,False
2,0,3,-0.336786,25.078313,0.020483,24.208610,-0.237448,20.733267,-0.281956,32.829720,...,18.870577,-0.051306,20.050884,-0.051306,20.050884,-0.092616,22.492845,0.139786,30.472674,False
3,0,4,0.571550,23.796227,0.170771,27.059930,-0.130328,23.920272,0.457751,34.066019,...,22.235792,0.214385,19.831823,0.214385,19.831823,0.136081,24.970415,0.347514,30.803236,False
4,0,5,-0.156618,28.030262,0.019398,28.681243,0.157825,25.146513,0.097109,42.098599,...,29.556387,-0.188304,21.520493,-0.188304,21.520493,-0.279403,28.959603,-0.032786,39.120852,False
5,0,6,0.486360,39.835651,0.217503,33.613739,0.122458,25.873762,0.156761,47.048994,...,35.851282,0.550318,22.058280,0.550318,22.058280,0.689219,33.831806,0.785863,37.863268,False
6,0,7,-0.493721,33.397087,0.464669,34.488052,0.721143,26.909594,-0.044024,45.575342,...,32.604831,0.026332,23.627218,0.026332,23.627218,-0.170747,35.702901,-1.110479,38.161832,False
7,0,8,0.617939,31.848664,0.029983,28.280461,-0.549961,26.226520,0.593422,37.852693,...,35.244524,0.500679,23.041928,0.500679,23.041928,0.197563,35.163717,0.608507,37.299374,False
8,0,9,0.113113,28.832383,-0.004529,29.117649,0.163063,24.426227,-0.083980,40.408078,...,30.023711,0.135233,21.409069,0.135233,21.409069,0.166556,26.039075,0.559385,32.210621,False
9,0,10,0.245758,39.515694,0.298746,40.212388,0.597114,25.445978,0.553471,54.590007,...,42.920719,0.141164,22.471298,0.141164,22.471298,0.098476,37.538723,-0.083512,37.897142,False


In [34]:
from source.data_import import save_pyarrow

save_pyarrow(df_features, file_name="feature_extracted")

/home/weasel/reps/ai-seizure-detectives/source/../data/feature_extracted.arrow was successfully written.
